In [6]:
import scipy
from scipy import signal
from scipy.io import wavfile

import librosa
import os
import wave
import sys
import numpy as np

import pickle
from sklearn.metrics import classification_report
from sklearn.mixture import GaussianMixture

import IPython.display as ipd

directory = 'G:\\Downloads\\ESC-50-master\\ESC-50-master\\audio'
model_filename = 'gmm_model_2.pkl'
filename = '1-19898-B-41.wav'

CHUNK = 1024
pre_emphasis = 0.97

test_X = []
test_y = []

def main():
    filepath = os.path.join(directory, filename)

    sample_rate, samples = wavfile.read(filepath)

    target_sample_rate = (16000/sample_rate)*len(samples) # ini rumus dr mana?
    samples = signal.resample(samples, int(target_sample_rate))

    # emphasized_signal
    samples = np.append(samples[0], samples[1:] - pre_emphasis * samples[:-1])
    
    window_size = 16000//2
    for idx in range(0, len(samples), window_size//2):
        window = samples[idx:idx+window_size]
        if len(window) < window_size:
            break

        # mfcc sound
        mfccs = librosa.feature.mfcc(window, n_mfcc=39, sr=sample_rate)
        mfccs_temp = mfccs.mean(axis=0)
        
        test_X.append(mfccs_temp)
        if filename.endswith("-41.wav"):
            test_y.append(1)
        else:
            test_y.append(0)
    
    ipd.display(ipd.Audio(filepath))

    # predict
    loaded_model = pickle.load(open(model_filename, 'rb'))
    result = loaded_model.predict(test_X)
    print("result: ")
    print(classification_report(test_y, result))

if __name__ == "__main__":
    main()

result: 
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         0
          1       0.00      0.00      0.00        19
          2       0.00      0.00      0.00         0
          3       0.00      0.00      0.00         0

avg / total       0.00      0.00      0.00        19



C:\Users\Zack\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Zack\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
